# Import thư viện

In [ ]:
import numpy as np
import os
import cv2
import h5py

import keras
from keras.models import Sequential
from keras.layers import ConvLSTM2D, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, TensorDataset

from convGRU import ConvGRU
import tensorflow_addons as tfa
from Yogi_torch import Yogi

# Dữ liệu đầu vào

### Đọc dữ liệu ảnh

In [ ]:
# Đường dẫn đến thư mục chứa ảnh
image_folder = r"E:\BacSon\Fair_2024_ConvLSTM_ConvGRU\data"
# image_folder = r"E:\BacSon\Fair_2023_ARIMA_LSTM_GRU\dataSet\data_img"

# Đọc từng file ảnh
image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

In [ ]:
# Chuyển ảnh về ma trận ảnh Gray

def matrix_images(image_folder, image_files):
    matrix_img = []
    for img_file in image_files:
        img_path = os.path.join(image_folder, img_file)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        matrix_img.append(image)

    return matrix_img

### Chuẩn hóa dữ liệu

In [ ]:
# Áp dụng Min-Max Scaling chuẩn hóa dữ liệu
def min_max_scaling(data):
    # Chuyển danh sách các mảng numpy thành một mảng numpy đa chiều
    data_array = np.array(data)

    # Tính giá trị min và max trên toàn bộ dữ liệu
    data_min = np.min(data_array)
    data_max = np.max(data_array)

    # Áp dụng công thức Min-Max Scaling
    scaled_data = (data_array - data_min) / (data_max - data_min)

    return scaled_data

### Tạo nhãn cho dữ liệu

In [ ]:
# Tạo nhãn chuỗi dữ liệu với 4 ảnh liên tiếp từ dữ liệu hình ảnh
def create_image_sequences(data, time_steps):
    num_samples, height, width, channels = data.shape
    num_frames = num_samples - time_steps
    input_sequences = np.zeros((num_frames, time_steps, height, width, channels))
    labels = np.zeros((num_frames, height, width, channels))  # Khởi tạo mảng label

    for i in range(num_frames):
        input_sequences[i] = data[i:i+time_steps]
        labels[i] = data[i+time_steps]  # Xác định label là frame tiếp theo

    return input_sequences, labels

### Chia dữ liệu

In [ ]:
# Chia dữ liệu
def Split_Data(X_data, y_data):
    size = int(len(X_data) * 0.8)
    size_val = int((len(X_data) - size) / 2)

    X_train = X_data[:size]
    X_val = X_data[size:size + size_val]
    X_test = X_data[-size_val:]

    y_train = y_data[:size]
    y_val = y_data[size:size + size_val]
    y_test = y_data[- size_val:]

    return X_train, y_train, X_val, y_val, X_test, y_test

# Mô hình ConvLSTM

In [ ]:
def conv_lstm(X_train, y_train, X_val, y_val, time_step, optimizer, save_best_model, epochs):
    height, width, channels = X_train.shape[2:]

    # Xây dựng mô hình ConvLSTM
    input_lstm = (time_step, height, width, channels)
    model_lstm = Sequential()
    model_lstm.add(ConvLSTM2D(filters=32, kernel_size=(3, 3), input_shape=input_lstm, padding='same', 
                              activation="tanh", return_sequences=True))
    model_lstm.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), padding='same', activation="tanh", return_sequences=True))
    model_lstm.add(ConvLSTM2D(filters=1, kernel_size=(3, 3), padding='same', activation="tanh"))

    # Compile mô hình
    model_lstm.compile(optimizer=optimizer, loss='mean_squared_error')

    # Dừng sớm
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

    # Tạo thư mục nếu chưa tồn tại
    os.makedirs(os.path.dirname(save_best_model), exist_ok=True)

    # Lưu lại model tốt nhất
    model_checkpoint = ModelCheckpoint(save_best_model, save_best_only=True, monitor='val_loss', mode='min', verbose=1)

    # Training model
    history = model_lstm.fit(X_train, y_train,
                    batch_size = 4,
                    epochs = epochs, 
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping, model_checkpoint])
    
    return model_lstm, history

# Mô hình Bi-ConvLSTM

In [ ]:
def conv_BiLSTM(X_train, y_train, X_val, y_val, time_step, optimizer, save_best_model, epochs):
    height, width, channels = X_train.shape[2:]

    # Xây dựng mô hình ConvLSTM
    input_bilstm = (time_step, height, width, channels)
    model_bilstm = Sequential()
    model_bilstm.add(Bidirectional(ConvLSTM2D(filters=16, kernel_size=(3, 3), input_shape=input_bilstm, 
                                              padding='same', activation="tanh", return_sequences=True)))
    model_bilstm.add(Bidirectional(ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='same', activation="tanh", return_sequences=True)))
    model_bilstm.add(Bidirectional(ConvLSTM2D(filters=1, kernel_size=(3, 3), padding='same', activation="tanh")))

    # Compile mô hình
    model_bilstm.compile(optimizer=optimizer, loss='mean_squared_error')

    # Dừng sớm
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

    # Tạo thư mục nếu chưa tồn tại
    os.makedirs(os.path.dirname(save_best_model), exist_ok=True)

    # Lưu lại model tốt nhất
    model_checkpoint = ModelCheckpoint(save_best_model, save_best_only=True, monitor='val_loss', mode='min', verbose=1)

    # Training model
    history = model_bilstm.fit(X_train, y_train,
                                batch_size = 4,
                                epochs = epochs, 
                                validation_data=(X_val, y_val),
                                callbacks=[early_stopping, model_checkpoint])
    
    return model_bilstm, history

# Mô hình ConvGRU

In [ ]:
def conv_gru(X_train):
    # Các tham số đầu vào
    input_size = (X_train.shape[-2:])
    channels = X_train.shape[2]
    num_layers = 3
    hidden_dim = [16, 128, 1]
    kernel_size = [(5, 5), (3, 3), (5, 5)]
    activation = torch.nn.ReLU

    # Xây dựng mô hình
    model_GRU = ConvGRU(input_size=input_size, input_dim=channels,
                        hidden_dim=hidden_dim, kernel_size=kernel_size, num_layers=num_layers,
                        dtype=torch.FloatTensor,
                        batch_first=True,
                        bias=True,
                        return_all_layers=False,
                        activation=activation)
    
    return model_GRU

In [ ]:
import torch.nn as nn

def save_model_to_hdf5(model, file_path):
    with h5py.File(file_path, 'w') as f:
        for name, param in model.named_parameters():
            f.create_dataset(name, data=param.data.cpu().numpy())

def train_ConvGRU(model, optimizer, train_dataloader, val_dataloader, best_model_path, num_epochs):
    criterion = nn.MSELoss()
    best_val_loss = float('inf')
    train_loss_epoch = []
    val_loss_epoch = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for X_batch, y_batch in train_dataloader:
            optimizer.zero_grad()
            outputs, _ = model(X_batch)
            output_last_layer = outputs[-1]
            loss = criterion(output_last_layer, y_batch)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        
        avg_train_loss = total_loss / len(train_dataloader)
        train_loss_epoch.append(avg_train_loss)
        
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_dataloader:
                outputs, _ = model(X_batch)
                output_last_layer = outputs[-1]
                val_loss = criterion(output_last_layer, y_batch)
                total_val_loss += val_loss.item()
        
        avg_val_loss = total_val_loss / len(val_dataloader)
        val_loss_epoch.append(avg_val_loss)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            save_model_to_hdf5(model, best_model_path)
            
    history_model = {"loss" : train_loss_epoch, "val_loss" : val_loss_epoch}
    return model, history_model

# Đánh giá và Test mô hình

### Biểu đồ train_loss và val_loss

In [ ]:
def plot_loss_model(history):
    # Kiểm tra loại của history để trích xuất giá trị loss phù hợp
    if isinstance(history, dict):
        train_loss = history['loss']
        val_loss = history['val_loss']
    else:
        train_loss = history.history['loss']
        val_loss = history.history['val_loss']

    # Vẽ biểu đồ
    plt.figure(figsize=(8, 5))
    plt.plot(train_loss, linestyle='-', label='Train loss')
    plt.plot(val_loss, linestyle='-', label='Val loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train loss và Val loss của mô hình')
    plt.yscale('log')  # Scale y-axis to logarithmic scale
    plt.legend()  # Thêm nhãn chú thích
    plt.show()

### So sánh chọn mô hình tốt nhất

In [ ]:
# Đánh giá mô hình trên tập test
def Compare_Adam_Yogi(model_Adam, model_Yogi, X_val, y_val):
    loss1 = model_Adam.evaluate(X_val, np.array(y_val))
    print(f'Test loss: {loss1}')

    loss2 = model_Yogi.evaluate(X_val, np.array(y_val))
    print(f'Test loss: {loss2}')

    if loss1 < loss2:
        print("Best Model Adam")
        model = model_Adam
    else:
        print("Best Model YoGi")
        model = model_Yogi

    return model

In [ ]:
def load_model_from_hdf5(file_path, model):
    with h5py.File(file_path, 'r') as f:
        for name, param in model.named_parameters():
            if name in f:
                param_data = torch.tensor(f[name][:])
                param.data.copy_(param_data)
            else:
                print(f"Không tìm thấy {name} trong file HDF5.")
    return model

def loss_model(X_train_gru, model_path, val_dataloader):
    model_eval = conv_gru(X_train_gru)
    # model_eval.load_state_dict(best_model_name)
    model_eval = load_model_from_hdf5(model_path, model_eval)
    model_eval.eval()

    criterion = torch.nn.MSELoss()
    # Tính toán tổng loss trên tập validation
    total_val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_dataloader:
            outputs, _ = model_eval(X_batch)
            output_last_layer = outputs[-1]
            val_loss = criterion(output_last_layer, y_batch)
            total_val_loss += val_loss.item()

    # Tính toán average validation loss
    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Average Validation Loss: {avg_val_loss}")

    return avg_val_loss

def Compare_Adam_Yogi_GRU(X_train_gru, model_path_Adam, model_path_Yogi, val_dataloader):
    loss_Adam = loss_model(X_train_gru, model_path_Adam, val_dataloader)
    loss_Yogi = loss_model(X_train_gru, model_path_Yogi, val_dataloader)

    if loss_Adam < loss_Yogi:
        print("Best Model Adam")
        best_model = model_path_Adam
    else:
        print("Best Model YoGi")
        best_model = model_path_Yogi

    return best_model

### Dự đoán 

In [ ]:
def test_model(model, X_test, y_test):
    # Dự đoán ảnh mới
    predicted = model.predict(X_test)
    predicted_images = np.array(predicted)
    y_test = np.array(y_test)

    # Đánh giá MSE MAE R^2
    mse = mean_squared_error(y_test.flatten(), predicted_images.flatten())
    print("Mean Squared Error (MSE):", mse)
    mae = mean_absolute_error(y_test.flatten(), predicted_images.flatten())
    print("Mean Absolute Error (MAE):", mae)
    r2 = r2_score(y_test.flatten(), predicted_images.flatten())
    print("R-squared (R2) Score:", r2)

    # Hiển thị các ảnh dự đoán
    fig, axs = plt.subplots(1, len(predicted_images), figsize=(15, 25))
    for i in range(len(predicted_images)):
        axs[i].imshow(predicted_images[i])  # Hiển thị ảnh cuối cùng trong mỗi mẫu dự đoán
        axs[i].set_title(f"Ảnh dự đoán {i+1}")
    plt.show()

In [ ]:
def test_model_BiConvLSTM(model, X_test, y_test):
    # Dự đoán ảnh mới
    predicted = model.predict(X_test)
    predicted_images = np.mean(predicted, axis=-1, keepdims=True)  # Tính trung bình hai kênh màu
    y_test = np.array(y_test)

    # Đánh giá MSE MAE R^2
    mse = mean_squared_error(y_test.flatten(), predicted_images.flatten())
    print("Mean Squared Error (MSE):", mse)
    mae = mean_absolute_error(y_test.flatten(), predicted_images.flatten())
    print("Mean Absolute Error (MAE):", mae)
    r2 = r2_score(y_test.flatten(), predicted_images.flatten())
    print("R-squared (R2) Score:", r2)

    # Hiển thị các ảnh dự đoán
    fig, axs = plt.subplots(1, len(predicted_images), figsize=(15, 25))
    for i in range(len(predicted_images)):
        axs[i].imshow(predicted_images[i])  # Hiển thị ảnh cuối cùng trong mỗi mẫu dự đoán
        axs[i].set_title(f"Ảnh dự đoán {i+1}")
    plt.show()

    return predicted_images

In [ ]:
def test_model_ConvGRU(X_train_gru, best_model_path, test_dataloader, y_test_gru):
    model_eval = conv_gru(X_train_gru)
    model_eval = load_model_from_hdf5(best_model_path, model_eval)
    model_eval.eval()

    predictions = []
    with torch.no_grad():
        for X_batch, y_batch in test_dataloader:
            outputs, _ = model_eval(X_batch)
            output_last_image = outputs[-1][:, -1, ...]
            predictions.append(output_last_image.cpu().numpy())

    predicted_images = np.concatenate(predictions)
    predicted_images = np.squeeze(predicted_images)

    mse = mean_squared_error(y_test_gru.flatten(), predicted_images.flatten())
    print("Mean Squared Error (MSE):", mse)
    mae = mean_absolute_error(y_test_gru.flatten(), predicted_images.flatten())
    print("Mean Absolute Error (MAE):", mae)
    r2 = r2_score(y_test_gru.flatten(), predicted_images.flatten())
    print("R-squared (R2) Score:", r2)

    fig, axs = plt.subplots(1, min(len(predicted_images), 5), figsize=(15, 25))
    for i in range(min(len(predicted_images), 5)):
        axs[i].imshow(predicted_images[i])
        axs[i].set_title(f"Ảnh dự đoán {i+1}")
    plt.show()

    return predicted_images

# Chạy các mô hình

### Chuẩn bị đầu vào

In [ ]:
# Đọc dữ liệu ảnh
data_img = matrix_images(image_folder, image_files)

# Chuẩn hóa dữ liệu
data_scaler = min_max_scaling(data_img)
data_lstm = np.expand_dims(data_scaler, axis=-1)

# Tạo chuỗi dữ liệu với 4 ảnh liên tiếp từ dữ liệu hình ảnh
epochs = 200
time_step = 4
X_data, y_data = create_image_sequences(data_lstm, time_step)

# Chia dữ liệu
X_train, y_train, X_val, y_val, X_test, y_test = Split_Data(X_data, y_data)

# Kiểm tra kích thước của X_train
print("Kích thước của X_train:", X_train.shape)
print("Kích thước của X_val:", X_val.shape)
print("Kích thước của X_test:", X_test.shape)

print("Kích thước của y_train:", y_train.shape)
print("Kích thước của y_val:", y_val.shape)
print("Kích thước của y_test:", y_test.shape)

In [ ]:
# Tạo chuỗi dữ liệu với 4 ảnh liên tiếp từ dữ liệu hình ảnh
data_gru = np.expand_dims(data_scaler, axis=1)
X_data_gru, y_data_gru = create_image_sequences(data_gru, time_step)

# Chia dữ liệu
X_train_gru, y_train_gru, X_val_gru, y_val_gru, X_test_gru, y_test_gru = Split_Data(X_data_gru, y_data_gru)

# Kiểm tra kích thước của X_train
print("Kích thước của X_train:", X_train_gru.shape)
print("Kích thước của X_val:", X_val_gru.shape)
print("Kích thước của X_test:", X_test_gru.shape)

print("Kích thước của y_train:", y_train_gru.shape)
print("Kích thước của y_val:", y_val_gru.shape)
print("Kích thước của y_test:", y_test_gru.shape)

# Chuyển đổi các mảng thành các tensor
X_train_tensor = torch.tensor(X_train_gru, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_gru, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_gru, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_gru, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_gru, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_gru, dtype=torch.float32)

# Tạo DataLoader cho training and validation
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_dataloader = DataLoader(test_dataset)

### Mô hình ConvLSTM

In [ ]:
# Training ConvLSTM bằng Adam
optimizer_ConvLSTM_Adam = keras.optimizers.Adam(learning_rate=1e-3)
save_model_ConvLSTM_Adam = './KetQua/ConvLSTM/best_model_ConvLSTM_Adam.hdf5'
model_ConvLSTM_Adam, history_ConvLSTM_Adam = conv_lstm(X_train, y_train, X_val, y_val, 
                                                       time_step, 
                                                       optimizer_ConvLSTM_Adam, 
                                                       save_model_ConvLSTM_Adam,
                                                       epochs)

In [ ]:
plot_loss_model(history_ConvLSTM_Adam)

In [ ]:
# Training ConvLSTM bằng Yogi
optimizer_ConvLSTM_Yogi = tfa.optimizers.Yogi(learning_rate=1e-3)
save_model_ConvLSTM_Yogi = './KetQua/ConvLSTM/best_model_ConvLSTM_Yogi.hdf5'
model_ConvLSTM_Yogi, history_ConvLSTM_Yogi = conv_lstm(X_train, y_train, X_val, y_val, 
                                                       time_step, 
                                                       optimizer_ConvLSTM_Yogi, 
                                                       save_model_ConvLSTM_Yogi,
                                                       epochs)

In [ ]:
plot_loss_model(history_ConvLSTM_Yogi)

In [ ]:
# Xác định mô hình tốt nhất
best_model_ConvLTSM = Compare_Adam_Yogi(model_ConvLSTM_Adam, model_ConvLSTM_Yogi, X_val, y_val)

# Dự đoán trên model tốt nhất
test_model(best_model_ConvLTSM, X_test, y_test)

### Mô hình Bi-ConvLSTM

In [ ]:
# Training Bi-ConvLSTM bằng Adam
optimizer_BiConvLSTM_Adam = keras.optimizers.Adam(learning_rate=1e-3)
save_model_BiConvLSTM_Adam = './KetQua/Bi-ConvLSTM/best_model_Bi-ConvLSTM_Adam.hdf5'
model_BiConvLSTM_Adam, history_BiConvLSTM_Adam = conv_BiLSTM(X_train, y_train, X_val, y_val, 
                                                            time_step, 
                                                            optimizer_BiConvLSTM_Adam, 
                                                            save_model_BiConvLSTM_Adam,
                                                            epochs)
plot_loss_model(history_BiConvLSTM_Adam)

In [ ]:
# Training Bi-ConvLSTM bằng Yogi
optimizer_BiConvLSTM_Yogi = tfa.optimizers.Yogi(learning_rate=1e-3)
save_model_BiConvLSTM_Yogi = './KetQua/Bi-ConvLSTM/best_model_Bi-ConvLSTM_Yogi.hdf5'
model_BiConvLSTM_Yogi, history_BiConvLSTM_Yogi = conv_BiLSTM(X_train, y_train, X_val, y_val, 
                                                            time_step, 
                                                            optimizer_BiConvLSTM_Yogi, 
                                                            save_model_BiConvLSTM_Yogi,
                                                            epochs)
plot_loss_model(history_BiConvLSTM_Yogi)

In [ ]:
# Xác định mô hình tốt nhất
best_model_BiConvLTSM = Compare_Adam_Yogi(model_BiConvLSTM_Adam, model_BiConvLSTM_Yogi, X_val, y_val)

# Dự đoán trên model tốt nhất
predicted_images_BiConvLTSM = test_model_BiConvLSTM(best_model_BiConvLTSM, X_test, y_test)

### Mô hình GRU

In [ ]:
# Training ConvGRU bằng Adam
model_ConvGRU_Adam = conv_gru(X_train_gru)
optimizer_ConvGRU_Adam = torch.optim.Adam(model_ConvGRU_Adam.parameters(), lr=1e-3)
save_model_ConvGRU_Adam = './KetQua/ConvGRU/best_model_ConvGRU_Adam.hdf5'

best_model_ConvGRU_Adam, history_ConvGRU_Adam = train_ConvGRU(model_ConvGRU_Adam, 
                                                            optimizer_ConvGRU_Adam, 
                                                            train_dataloader, 
                                                            val_dataloader, 
                                                            save_model_ConvGRU_Adam,
                                                            epochs)

plot_loss_model(history_ConvGRU_Adam)

In [ ]:
# Training ConvGRU bằng Yogi
model_ConvGRU_Yogi = conv_gru(X_train_gru)
optimizer_ConvGRU_Yogi = Yogi(model_ConvGRU_Yogi.parameters(), lr=1e-3)
save_model_ConvGRU_Yogi = './KetQua/ConvGRU/best_model_ConvGRU_Yogi.hdf5'

best_model_ConvGRU_Yogi, history_ConvGRU_Yogi = train_ConvGRU(model_ConvGRU_Yogi, 
                                                            optimizer_ConvGRU_Yogi, 
                                                            train_dataloader, 
                                                            val_dataloader,
                                                            save_model_ConvGRU_Yogi,
                                                            epochs)
plot_loss_model(history_ConvGRU_Yogi)

In [ ]:
# Xác định mô hình tốt nhất
best_model_path_ConvGRU = Compare_Adam_Yogi_GRU(X_train_gru, save_model_ConvGRU_Adam, save_model_ConvGRU_Yogi, val_dataloader)

# Dự đoán trên model tốt nhất
predictions = test_model_ConvGRU(X_train_gru, best_model_path_ConvGRU, test_dataloader, y_test_gru)